# GoogLeNet
前面我们讲的 VGG 是 2014 年 ImageNet 比赛的亚军，那么冠军是谁呢？就是我们马上要讲的 GoogLeNet，这是 Google 的研究人员提出的网络结构，在当时取得了非常大的影响，因为网络的结构变得前所未有，它颠覆了大家对卷积网络的串联的印象和固定做法，采用了一种非常有效的 inception 模块，得到了比 VGG 更深的网络结构，但是却比 VGG 的参数更少，因为其去掉了后面的全连接层，所以参数大大减少，同时有了很高的计算效率。

![](https://ws2.sinaimg.cn/large/006tNc79ly1fmprhdocouj30qb08vac3.jpg)

这是 googlenet 的网络示意图，下面我们介绍一下其作为创新的 inception 模块。

## Inception 模块
在上面的网络中，我们看到了多个四个并行卷积的层，这些四个卷积并行的层就是 inception 模块，可视化如下

![](https://ws4.sinaimg.cn/large/006tNc79gy1fmprivb2hxj30dn09dwef.jpg)


一个 inception 模块的四个并行线路如下：<br>
1.一个 1 x 1 的卷积，一个小的感受野进行卷积提取特征<br>
2.一个 1 x 1 的卷积加上一个 3 x 3 的卷积，1 x 1 的卷积降低输入的特征通道，减少参数计算量，然后接一个 3 x 3 的卷积做一个较大感受野的卷积<br>
3.一个 1 x 1 的卷积加上一个 5 x 5 的卷积，作用和第二个一样<br>
4.一个 3 x 3 的最大池化加上 1 x 1 的卷积，最大池化改变输入的特征排列，1 x 1 的卷积进行特征提取

最后将四个并行线路得到的特征在通道这个维度上拼接在一起，下面我们可以实现一下

In [1]:
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torchvision.datasets import CIFAR10

In [2]:
# 定义 conv + relu + BN 作为基本层结构
def conv_relu(in_channel, out_channel, kernel, stride=1, padding=0):
    layer = nn.Sequential(
        nn.Conv2d(in_channel, out_channel, kernel, stride, padding),
        nn.BatchNorm2d(out_channel, eps=1e-3),   # 为了数值稳定，在分母添加的值  默认为1e-5
        nn.ReLU(True)
    )
    return layer

In [3]:
# 定义 Inception 结构
class inception(nn.Module):
    def __init__(self, in_channel, out1_1, out2_1, out2_3, out3_1, out3_5, out4_1):
        super(inception, self).__init__()
        # 第一条线路
        self.branch1x1 = conv_relu(in_channel, out1_1, 1)
        
        # 第二条线路
        self.branch3x3 = nn.Sequential(
            conv_relu(in_channel, out2_1, 1),
            conv_relu(out2_1, out2_3, 3, padding=1)
        )
        
        # 第三条线路
        self.branch5x5 = nn.Sequential(
            conv_relu(in_channel, out3_1, 1),
            conv_relu(out3_1, out3_5, 5, padding=2)
        )
        
        # 第四条线路
        self.branch_pool = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            conv_relu(in_channel, out4_1, 1)
        )
        
    def forward(self, x):
        f1 = self.branch1x1(x)
        f2 = self.branch3x3(x)
        f3 = self.branch5x5(x)
        f4 = self.branch_pool(x)
        output = torch.cat((f1, f2, f3, f4), dim=1)
        return output

In [4]:
# 测试Inception 对不对
test_net = inception(3, 64, 48, 128, 64, 96, 32)   # 除了第一个值 需要注意下  in_channel  其他的随意定义  因为是改变输出的通道数
test_x = Variable(torch.zeros(3, 3, 96, 96))    # 第二个参数得和上面第一个参数保持一致
print('input shape: {} x {} x {}'.format(test_x.shape[1], test_x.shape[2], test_x.shape[3]))
test_y = test_net(test_x)
print('output shape: {} x {} x {}'.format(test_y.shape[1], test_y.shape[2], test_y.shape[3]))

input shape: 3 x 96 x 96
output shape: 320 x 96 x 96


可以看到输入经过了inception模块后， 大小没有变化，通道的维度变多了

In [5]:
# 下面定义GoogLet， 可以看作是很多很多个inception模块的串联，注意，原论文中使用了多个输出来解决梯度消失的问题，
# 这里我们只定义一个简单版本的GoogLetNet，简化为一个输出

class googlenet(nn.Module):
    
    def __init__(self, in_channel, num_classes, verbose=False):
        super(googlenet, self).__init__()
        self.verbose = verbose
        
        self.block1 = nn.Sequential(
            conv_relu(in_channel, out_channel=64, kernel=7, stride=2, padding=3),
            nn.MaxPool2d(3, 2)
        )
        
        self.block2 = nn.Sequential(
            conv_relu(64, 64, kernel=1),
            conv_relu(64, 192, kernel=3, padding=1),
            nn.MaxPool2d(3, 2)
        )
        
        self.block3 = nn.Sequential(
            inception(192, 64, 96, 128, 16, 32, 32),
            inception(256, 128, 128, 192, 32, 96, 64),
            nn.MaxPool2d(3, 2)
        )
        
        self.block4 = nn.Sequential(
            inception(480, 192, 96, 208, 16, 48, 64),
            inception(512, 160, 112, 224, 24, 64, 64),
            inception(512, 128, 128, 256, 24, 64, 64),
            inception(512, 112, 144, 288, 32, 64, 64),
            inception(528, 256, 160, 320, 32, 128, 128),
            nn.MaxPool2d(3, 2)
        )
        
        self.block5 = nn.Sequential(
            inception(832, 256, 160, 320, 32, 128, 128),
            inception(832, 384, 182, 384, 48, 128, 128),
            nn.AvgPool2d(2)
        )

        self.classifier = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        x = self.block1(x)
        if self.verbose:
            print('block 1 output: {}'.format(x.shape))
        x = self.block2(x)
        if self.verbose:
            print('block 2 output: {}'.format(x.shape))
        x = self.block3(x)
        if self.verbose:
            print('block 3 output: {}'.format(x.shape))
        x = self.block4(x)
        if self.verbose:
            print('block 4 output: {}'.format(x.shape))
        x = self.block5(x)
        if self.verbose:
            print('block 5 output: {}'.format(x.shape))
        x = x.view(x.shape[0], -1)
        x = self.classifier(x)
        return x

In [6]:
test_net = googlenet(3, 10, True)
test_x = Variable(torch.zeros(1, 3, 96, 96))
test_y = test_net(test_x)
print('output: {}'.format(test_y.shape))

block 1 output: torch.Size([1, 64, 23, 23])
block 2 output: torch.Size([1, 192, 11, 11])
block 3 output: torch.Size([1, 480, 5, 5])
block 4 output: torch.Size([1, 832, 2, 2])
block 5 output: torch.Size([1, 1024, 1, 1])
output: torch.Size([1, 10])


In [7]:
from utils import train

def data_tf(x):
    x = x.resize((96, 96), 2) # 将图片放大到 96 x 96
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5 # 标准化
    x = x.transpose((2, 0, 1)) # 将 channel 放到第一维，只是 pytorch 要求的输入方式
    x = torch.from_numpy(x)
    return x
     
train_set = CIFAR10('./data', train=True, transform=data_tf)
train_data = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_set = CIFAR10('./data', train=False, transform=data_tf)
test_data = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False)

net = googlenet(3, 10)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [8]:
#  训练
train(net, train_data, test_data, 20, optimizer, criterion)

E:\dev\Jupyter\深度学习入门之PyTorch\PyTorch-from-zero-to-one\utils.py:52: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  im = Variable(im.cuda(), volatile=True)
E:\dev\Jupyter\深度学习入门之PyTorch\PyTorch-from-zero-to-one\utils.py:53: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  label = Variable(label.cuda(), volatile=True)


Epoch 0. Train Loss: 1.529391, Train Acc: 0.440117, Valid Loss: 1.685073, Valid Acc: 0.419403, Time 00:01:35
Epoch 1. Train Loss: 1.062369, Train Acc: 0.622842, Valid Loss: 1.323546, Valid Acc: 0.552017, Time 00:01:42
Epoch 2. Train Loss: 0.844526, Train Acc: 0.704364, Valid Loss: 1.322599, Valid Acc: 0.564280, Time 00:01:42
Epoch 3. Train Loss: 0.692204, Train Acc: 0.758712, Valid Loss: 1.243304, Valid Acc: 0.603441, Time 00:01:43
Epoch 4. Train Loss: 0.579017, Train Acc: 0.800412, Valid Loss: 0.807383, Valid Acc: 0.721025, Time 00:01:42
Epoch 5. Train Loss: 0.483130, Train Acc: 0.833560, Valid Loss: 0.922476, Valid Acc: 0.693829, Time 00:01:42
Epoch 6. Train Loss: 0.406008, Train Acc: 0.859595, Valid Loss: 0.791454, Valid Acc: 0.751088, Time 00:01:41
Epoch 7. Train Loss: 0.345969, Train Acc: 0.880994, Valid Loss: 1.108616, Valid Acc: 0.667227, Time 00:01:41
Epoch 8. Train Loss: 0.287470, Train Acc: 0.902953, Valid Loss: 0.797404, Valid Acc: 0.752769, Time 00:01:42
Epoch 9. Train Loss

GoogLeNet 加入了更加结构化的 Inception 块使得我们能够使用更大的通道，更多的层，同时也控制了计算量。

**GoogLeNet 有很多后续的版本，尝试看看论文，看看有什么不同，实现一下：  
v1：最早的版本  
v2：加入 batch normalization 加快训练  
v3：对 inception 模块做了调整  
v4：基于 ResNet 加入了 残差连接  **